# Attention mechanism 

- Seq2Seq 모델의 문제점
    - Seq2Seq 모델은 Encoder에서 입력 시퀀스에 대한 특성을 **하나의 고정된 context vector**에 압축하여 Decoder로 전달 한다. Decoder는 이 context vector를 이용해서 출력 시퀀스를 만든다.
    - 하나의 고정된 크기의 vector에 모든 입력 시퀀스의 정보를 넣다보니 정보 손실이 발생한다.
    - Decoder에서 출력 시퀀스를 생성할 때 동일한 context vector를 기반으로 한다. 그러나 각 생성 토큰마다 입력 시퀀스에서 참조해야 할 중요도가 다를 수 있다. seq2seq는 encoder의 마지막 hidden state를 context로 받은 뒤 그것을 이용해 모든 출력 단어들을 생성하므로 그 중요도에 대한 반영이 안된다.

## Attention Mechanism 아이디어
-  Decoder에서 출력 단어를 예측하는 매 시점(time step)마다, Encoder의 입력 문장(context vector)을 다시 참고 하자는 것. 이때 전체 입력 문장의 단어들을 동일한 비율로 참고하는 것이 아니라, Decoder가 해당 시점(time step)에서 예측해야할 단어와 연관이 있는 입력 부분을 좀 더 집중(attention)해서 참고 할 수 있도록 하자는 것이 기본 아이디어이다.
- 다양한 Attention 종류들이 있다.
    -  Decoder에서 출력 단어를 예측하는 매 시점(time step)마다 Encoder의 입력 문장의 어느 부분에 더 집중(attention) 할지를 계산하는 방식에 따라 다양한 attention 기법이 있다.
    -  `dot attention - Luong`, `scaled dot attention - Vaswani`, `general  attention - Luong`, `concat  attention - Bahdanau` 등이 있다.

# Data Loading

In [1]:
import os
import numpy as np
import pandas as pd
import random

df = pd.read_csv('data/chatbot_data.csv')
df.head()

,Q,A,label
0,12시 땡!,하루가 또 가네요.,0
1,1지망 학교 떨어졌어,위로해 드립니다.,0
2,3박4일 놀러가고 싶다,여행은 언제나 좋죠.,0
3,3박4일 정도 놀러가고 싶다,여행은 언제나 좋죠.,0
4,PPL 심하네,눈살이 찌푸려지죠.,0


In [2]:
# 라벨 제거
df.drop(columns='label', inplace=True)

# 토큰화

In [3]:
## 질문들 + 답변들 합쳐서 학습.
question_texts = df['Q']
answer_texts = df['A']
all_texts = list(question_texts + " "+answer_texts) # 같은 index끼리 합치기 => list로 변환
len(question_texts), len(answer_texts), len(all_texts)

(11823, 11823, 11823)

## Tokenizer 학습

In [4]:
from tokenizers import Tokenizer
from tokenizers.models import BPE
from tokenizers.pre_tokenizers import Whitespace
from tokenizers.trainers import BpeTrainer

vocab_size = 10_000
min_frequency = 5 

tokenizer = Tokenizer(BPE(unk_token="[UNK]"))
tokenizer.pre_tokenizer = Whitespace()
trainer = BpeTrainer(
    vocab_size=vocab_size,
    min_frequency=min_frequency,
    continuing_subword_prefix='##',            # 연결 subword 앞에 붙일 접두어지정. 
    special_tokens=["[PAD]", "[UNK]", "[SOS]", "[EOS]"] 
    # [SOS]: 문장의 시작을 의미하는 토큰. [EOS]: 문장이 끝난 것을 표시.
)
# tokenizer: token + ##izer
## 학습
tokenizer.train_from_iterator(all_texts, trainer=trainer) # 리스트로 부터 학습
## tokenizer.train("파일경로") # 파일에 있는 text를 학습.

In [5]:
print("총 어휘수:", tokenizer.get_vocab_size())

총 어휘수: 7041


## 저장

In [6]:
dir_path = "saved_model/vocab"
os.makedirs(dir_path, exist_ok=True)
vocab_path = os.path.join(dir_path, "chatbot_attn_bpe.json")
tokenizer.save(vocab_path)

# Dataset 생성
- 한문장 단위로 학습시킬 것이므로 DataLoader를 생성하지 않고 Dataset에서 index로 조회한 질문-답변을 학습시킨다.

In [7]:
import random
import os
import time
import math

import numpy as np

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torch import optim

device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cpu'

### Dataset 클래스 정의

In [9]:
class ChatbotDataset(Dataset):

    """
    Attribute
        max_length
        tokenizer: Tokenizer
        vocab_size: int - Tokenizer에 등록된 총 어휘수
        SOS: int - [SOS] 문장의 시작 토큰 id
        EOS: int = [EOS] 문장의 끝 토큰 id
        question_squences: list - 모든 질문 str을 token_id_list(token sequence) 로 변환하여 저장한 list 
        answser_sequences: list - 모든 답변 str을 token_id_list(token sequence) 로 변환하여 저장한 list.
    """
    def __init__(self, question_texts, answer_texts, tokenizer, min_length=3, max_length=25):
        """
        question_texts: list[str] - 질문 texts 목록. 리스트에 질문들을 담아서 받는다. ["질문1", "질문2", ...]
        answer_texts: list[str] - 답 texts 목록. 리스트에 답변들을 담아서 받는다.     ["답1", "답2", ...]
        tokenizer: Tokenizer
        min_length=3: int - 최소 토큰 개수. 질문과 답변의 token수가 min_length 이상인 것만 학습한다.
        max_length=25:int 개별 댓글의 token 개수. 모든 댓글의 토큰수를 max_length에 맞춘다.
        """
        self.min_length = min_length
        self.max_length = max_length
        self.tokenizer = tokenizer
        
        self.vocab_size = tokenizer.get_vocab_size()
        self.SOS = self.tokenizer.token_to_id('[SOS]')
        self.EOS = self.tokenizer.token_to_id('[EOS]')
        # 질문/답변을 토큰으로 변환한 list
        self.question_sequences = []
        self.answer_sequences = []
        for q, a in zip(question_texts, answer_texts):
            q_token = self.__process_sequence(q)
            a_token = self.__process_sequence(a)
            if len(q_token) > min_length and len(a_token) > min_length:
                self.question_sequences.append(q_token)
                self.answer_sequences.append(a_token)

    def __add_special_tokens(self, token_sequence):
        """
        max_length 보다 token 수가 많은 경우 max_length에 맞춰 뒤를 잘라낸다.
        token_id_list 에 [EOS] 토큰 추가. 
        """
        # max_length 보다 큰 경우 줄이기. (적은 경우에는 그냥 유지.)
        token_id_list = token_sequence[:self.max_length-1] # EOS 토큰 포함해서 max_length가 되게 하기 위해서 -1
        token_id_list.append(self.EOS) # 마지막 토큰으로 [EOS] 추가.

        return token_id_list

    
    def __process_sequence(self, text):
        """
        한 문장 string을 받아서 token_id 리스트(list)로 변환 후 반환
        """
        encode = self.tokenizer.encode(text)
        token_ids = self.__add_special_tokens(encode.ids)
        return token_ids
    
    def __len__(self):
        return len(self.question_sequences)

    def __getitem__(self, index):        
        q = torch.tensor(self.question_sequences[index], dtype=torch.int64).unsqueeze(1) # [seq_length, 1]
        a = torch.tensor(self.answer_sequences[index], dtype=torch.int64).unsqueeze(1)
        return q, a

### Dataset 객체 생성

In [10]:
MAX_LENGTH = 20
MIN_LENGTH = 2
dataset = ChatbotDataset(question_texts, answer_texts, tokenizer, MIN_LENGTH, MAX_LENGTH)
print(len(dataset))

11714


In [11]:
a, b = dataset[0]

In [12]:
b

tensor([[6119],
        [ 379],
        [  48],
        [2253],
        [   9],
        [   3]])

# 모델

## Encoder
- seq2seq 모델과 동일 한 구조
    - 이전 코드(seq2seq)와 비교해서 forward()에서 입력 처리는 token 하나씩 하나씩 처리한다. 

![encoder](figures/attn_encoder-network_graph.png)

In [13]:
class Encoder(nn.Module):
    
    def __init__(self, num_vocabs, hidden_size, embedding_dim, num_layers):
        """
        Parameter
            num_vocabs: int - 총 어휘수 
            hidden_size: int - GRU의 hidden size
            embedding_dim: int - Embedding vector의 차원수 
            num_layers: int - GRU의 layer수
        """
        super().__init__()
        self.num_vocabs = num_vocabs
        self.hidden_size = hidden_size
        
        # 임베딩 레이어
        self.embedding = nn.Embedding(num_vocabs, embedding_dim)
        # GRU
        self.gru = nn.GRU(embedding_dim, hidden_size, num_layers=num_layers) # 단방향.(한단어씩 처리하므로)

    def forward(self, x, hidden):
        """
        질문의 token한개의 토큰 id를 입력받아 hidden state를 출력
        Parameter
            x: 한개 토큰. shape-[1]
            hidden: hidden state (이전 처리결과). shape: [1, 1, hidden_size]
        Return
            tuple: (output, hidden) - output: [1, 1, hidden_size],  hidden: [1, 1, hidden_size]
        """
        # x: [batch, 1]
        x = self.embedding(x).unsqueeze(0) # (1, embedding_dim) -> (1, 1, embedding_dim)\
        out, hidden = self.gru(x, hidden)
        # out: (seq_len, batch, hidden_size), hidden: (1, batch, hidden_size)
        return out, hidden
    
    def init_hidden(self, device):
        """
        처음 timestep에서 입력할 hidden_state. 
        값: 0
        shape: (Bidirectional(1) x number of layers(1), batch_size: 1, hidden_size) 
        """
        return torch.zeros(1, 1, self.hidden_size, device=device)

## Attention 적용 Decoder
![seq2seq attention outline](figures/attn_seq2seq_attention_outline.png)

- Attention은 Decoder 네트워크가 순차적으로 다음 단어를 생성하는 자기 출력의 모든 단계에서 인코더 출력 중 연관있는 부분에 **집중(attention)** 할 수 있게 한다. 
- 다양한 어텐션 기법중에 **Luong attention** 방법은 다음과 같다.
  
![attention decoder](figures/attn_decoder-network_graph.png)

### Attetion Weight
- Decoder가 현재 timestep의 단어(token)을 생성할 때 Encoder의 output 들 중 어떤 단어에 좀더 집중해야 하는지 계산하기 위한 가중치값.
  
![Attention Weight](figures/attn_attention_weight.png)

### Attention Value
- Decoder에서 현재 timestep의 단어를 추출할 때 사용할 Context Vector. 
    - Encoder의 output 들에 Attention Weight를 곱한다.
    - Attention Value는 Decoder에서 단어를 생성할 때 encoder output의 어떤 단어에 더 집중하고 덜 집중할지를 가지는 값이다.

![attention value](figures/attn_attention_value.png)

### Feature Extraction
- Decoder의 embedding vector와 Attention Value 를 합쳐 RNN(GRU)의 입력을 만든다.
    - **단어를 생성하기 위해 이전 timestep에서 추론한 단어(현재 timestep의 input)** 와 **Encoder output에 attention이 적용된 값** 이 둘을 합쳐 입력한다.
    - 이 값을 Linear Layer함수+ReLU를 이용해 RNN input_size에 맞춰 준다. (어떻게 input_size에 맞출지도 학습시키기 위해 Linear Layer이용)

![rnn](figures/att_attention_combine.png)

### 단어 예측(생성)
- RNN에서 찾은 Feature를 총 단어개수의 units을 출력하는 Linear에 입력해 **다음 단어를 추론한다.**
- 추론한 단어는 다음 timestep의 입력($X_t$)으로 RNN의 hidden은 다음 timestep 의 hidden state ($h_{t-1}$) 로 입력된다.


In [44]:
class AttentionDecoder(nn.Module):

    def __init__(self, num_vocabs, hidden_size, embedding_dim, dropout_p, max_length):
        super().__init__()
        self.num_vocabs=num_vocabs # 총 단어수 (vocab size)
        self.hidden_size = hidden_size
        self.max_length = max_length

        # embedding layer
        self.embedding = nn.Embedding(num_vocabs, embedding_dim)

        # attention weight를 계산하는 Linear: out_features: max_length - encoder outputs의 최대개수.
        self.attn = nn.Linear(hidden_size+embedding_dim,  max_length)    
        # attention value 와 현재 timestep의 embedding vector를 합친것을 받아서 
        # gru의 input을 만드는 Linear.
        self.attn_combine = nn.Linear(hidden_size+embedding_dim , hidden_size)

        self.dropout = nn.Dropout(dropout_p)
        # gru
        self.gru = nn.GRU(hidden_size, hidden_size)
        # 분류기(다음 단어 예측기)
        self.classifier = nn.Linear(hidden_size, num_vocabs)

    def forward(self, x, hidden, encoder_outputs):
        """
        Parameter
            x: 현재 timestep의 입력 토큰(단어) id
            hidden: 이전 timestep 처리결과 hidden state
            encoder_outputs: Encoder output들. 
        Return
            tupe: (output, hidden, attention_weight)
                output: 단어별 다음 단어일 확률.  shape: [vocab_size]
                hidden: hidden_state. shape: [1, 1, hidden_size]
                atttention_weight: Encoder output 중 어느 단어에 집중해야하는 지 가중치값. shape: [1, max_length]
        
        현재 timestep 입력과 이전 timestep 처리결과를 기준으로 encoder_output와 계산해서  encoder_output에서 집중(attention)해야할 attention value를 계산한다.
        attention value와 현재 timestep 입력을 기준으로 단어를 추론(생성) 한다.
        """
        ## embedding 작업
        # shape:   입력 - [1](단어 한개), 출력: [1, embeding_dim] -> [1, 1, embedding_dim]
        embedding = self.embedding(x).unsqueeze(0)
        embedding = self.dropout(embedding)
        
        # attention weight 계산 입력값. embedding 와 hidden_state 합치기.
        ## pytorch에서 tensor 합치는 함수. torch.concat((합칠대상들, ))
        attn_in = torch.concat((embedding[0], hidden[0]), dim=1)  
        # shape: (1, embedding_dim + hidden_size) -> attn Linear의 입력.
        attn_score = self.attn(attn_in) # shape (1, max_length)
        attn_weight = nn.Softmax(dim=-1)(attn_score)

        # attention value를 계산. (어떤 encoder의 토큰에 더 집중할지 집중도값.)
        ## attn_weigth x encoder의 output
        attn_value = torch.bmm(            
            attn_weight.unsqueeze(0), # 배치축 추가. (1, max_length)->(1,  1,          max_length)
            encoder_outputs.unsqueeze(0) # (max_length, hidden_size)->(1, max_legnth, hidden_size)
        )
        # attn_value: (1, 1, hidden_size)
    
        ### attn_combine: attn_value + embedding_vector (합친것-concatenate)
        ##### GRU의 현재 timestep의 입력값.
        attn_combine_in = torch.concat([
            attn_value[0], embedding[0]
            # [1, hidden_size]+[1, embedding_dim] = [1, h_s + e_dim]
        ], dim=1)  # 합치는 축 방향 지정. h+e을 합치므로 1축 지정.
        gru_in = self.attn_combine(attn_combine_in)# 출력: [1, hidden_size]
        gru_in = gru_in.unsqueeze(0) # [1, 1, hidden_size] : seq_len 축 추가.
        gru_in = nn.ReLU()(gru_in)
        #### GRU에 현재 timestep값 + attention value 넣어서 hidden_state출력(feature vec.)
        out, hidden_state = self.gru(gru_in, hidden)

        #### classifier에 out을 입력해서 다음 단어를 예측
        last_out = self.classifier(out[0]) 
        #out: [1, 1, hidden]->[1:batch, hidden] -> last_out: [1, num_vocabs]

        return last_out[0], hidden_state, attn_weight
    
    def initHidden(self, device):
        # 첫번째 timestep에 넣어줄 hidden state를 생성. (0)
        return torch.zeros(1, 1, self.hidden_size, device=device)

In [45]:
a = torch.randn(3, 4, 5)
b = torch.randn(3, 5, 2)
c = torch.bmm(a, b)
c.shape

torch.Size([3, 4, 2])

In [46]:
dummy_decoder = AttentionDecoder(
    num_vocabs=tokenizer.get_vocab_size(), 
    hidden_size=256,
    embedding_dim=200,
    dropout_p=0.3,
    max_length=20
)
dummy_encoder = Encoder(
    num_vocabs=tokenizer.get_vocab_size(),
    hidden_size=256,
    embedding_dim=200,
    num_layers=1
)

In [47]:
x, y = dataset[0]
e_hidden = dummy_encoder.init_hidden(device)
# e_hidden.shape
e_o, e_h = dummy_encoder(x[0], e_hidden)  # 한개 문장의 첫번째(한개) 토큰
print(e_o.shape, e_h.shape)

torch.Size([1, 1, 256]) torch.Size([1, 1, 256])


In [48]:
dummy_encoder_outputs = torch.randn(20, 256)  # 20: seq length, 256: hidden size
d_hidden = dummy_decoder.initHidden(device)
# d_hidden.shape
word, d_h, attn_weight = dummy_decoder(y[0], d_hidden, dummy_encoder_outputs)

In [51]:
print(word.shape)
print(d_h.shape)
print(attn_weight.shape)

torch.Size([7041])
torch.Size([1, 1, 256])
torch.Size([1, 20])


In [52]:
attn_weight

tensor([[0.0380, 0.0158, 0.0688, 0.0251, 0.0828, 0.0530, 0.0282, 0.0636, 0.0682,
         0.0695, 0.0218, 0.0324, 0.0434, 0.0757, 0.0279, 0.0257, 0.0459, 0.0984,
         0.0424, 0.0732]], grad_fn=<SoftmaxBackward0>)

# Training

In [54]:
SOS_TOKEN = dataset.tokenizer.token_to_id("[SOS]")
EOS_TOKEN = dataset.tokenizer.token_to_id("[EOS]")

In [ ]:
# 한개 question-answer set 학습
def train(input_tensor:"질문한개", target_tensor:"답변한개", encoder, decoder, 
          encoder_optimizer, decoder_optimizer, 
          loss_fn, device, max_length:"한문장의 토큰수", teacher_forcing_ratio=0.9):

    loss = 0.0 # loss값을 저장할 변수.
    
    # 옵티마이저 초기화 (파라미터 초기화)
    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()
    
    ###### Encoder 처리
    encoder_hidden = encoder.initHidden(device) # 첫 timestep에 넣을 hidden state값
    # 질문/답변의 length(토큰수) 조회
    input_length = input_tensor.shape[0]
    output_length = target_tensor.shape[0]

    encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)
    ########## 질문 문장의 hidden state를 토큰별로 계산해서 encoder_outputs에 저장.
    for e_index in range(input_length):
        encoder_out, encoder_hidden = encoder(input_tensor[e_index], encoder_hidden)
        encoder_outputs[e_index] = encoder_out

    ########### Decoder 처리
    # 첫번째 timestep 입력 토큰: [SOS] 
    decoder_input = torch.tensor([SOS_TOKEN], device=device)
    decoder_hidden = encoder_hidden  # 첫번째 hidden_state: encoder의 마지막 hidden state
    teacher_forcing = True if teacher_forcing_ratio > random.random() else False
    ##### 한개 단어씩 생성.
    for d_index in range(output_length):
        decoder_out, decoder_hidden, attn_weight = decoder(decoder_input, 
                                                           decoder_hidden, 
                                                           encoder_outputs)
        # loss계산
        loss = loss + loss_fn(decoder_out.unsqueeze(0), # [총단어수] -> [1, 총단어수]
                              target_tensor[d_index])
        #### 다음 timestep에 넣을 input 토큰 생성.
        ##### teacher_forcing: True-정답 단어, False: 모델 추론 단어. ==> decoder_input에 대입
        if teacher_forcing:
            decoder_input = target_tensor[d_index]
        else:
            output_token = decoder_out.argmax(dim=-1).unsqueeze(0)
            decoder_input = output_token.detach() # 역전파할 때 grad 계산 대상에서 빼기.
        
        ## teacher_forcing_rate를 업데이트 (학습이 진행될 수록 줄여준다.)
        teacher_forcing_ratio = teacher_forcing_ratio * 0.99
        if decoder_input == EOS_TOKEN: # 찾은 토큰이 문장의 끝. 멈추기
            break
    ### 순전파 완료 -> 역전파 grad계산 및 파라미터 업데이트
    loss.backward()
    encoder_optimizer.step()
    decoder_optimizer.step()

    # loss 평균 리턴
    return loss.item() / output_length
    

In [ ]:
def train_iterations(encoder, decoder, n_iters, dataset, device, log_interval=1000, learning_rate=0.001):
    
    pass

  

## Model 생성, 학습

## 저장

## 검증